# MapD Charting Example with Altair

Let's see if we can replicate [this](https://omnisci.github.io/mapd-charting/example/example1.html) MapD charting example in Python with Altair, Vega Lite, and Vega:

![](https://cloud.githubusercontent.com/assets/2932405/25641647/1acce1f2-2f4a-11e7-87d4-a4e80cb262f5.gif)

In [7]:
import altair as alt
import jupyterlab_omnisci.vega_ibis
import ibis

alt.data_transformers.enable('ibis')
alt.renderers.enable('ibis')

RendererRegistry.enable('ibis')

In [8]:
from altair_transform.utils import evaljs


In [9]:
conn = ibis.mapd.connect(
    host='metis.mapd.com', user='mapd', password='HyperInteractive',
    port=443, database='mapd', protocol= 'https'
)
t = conn.table("flights_donotmodify")

In [10]:
jupyterlab_omnisci.vega_ibis.FALLBACK = False

alt.Chart(t[t.dep_timestamp, t.depdelay]).transform_filter(
    'datum.dep_timestamp != null'
).mark_line().encode(
    alt.X(
        'yearmonthdate(dep_timestamp):T',
    ),
    alt.Y(
        'count():Q',
        scale=alt.Scale(zero=False)
    )
)

Chart({
  data: Empty DataFrame
  Columns: [dep_timestamp, depdelay]
  Index: [],
  encoding: FacetedEncoding({
    x: X({
      field: 'dep_timestamp',
      timeUnit: 'yearmonthdate',
      type: 'temporal'
    }),
    y: Y({
      aggregate: 'count',
      scale: Scale({
        zero: False
      }),
      type: 'quantitative'
    })
  }),
  mark: 'line',
  transform: [FilterTransform({
    filter: 'datum.dep_timestamp != null'
  })]
})

In [11]:
states = alt.selection_multi(fields=['origin_state'])
airlines = alt.selection_multi(fields=['carrier_name'])

flights_by_state = alt.Chart(
    t[t.origin_state, t.carrier_name],
    title="Total Number of Flights by State"
).transform_filter(
    airlines
).mark_bar().encode(
    x='count()',
    y=alt.Y('origin_state', sort=alt.Sort(encoding='x', order='descending')),
    color=alt.condition(states, alt.ColorValue("steelblue"), alt.ColorValue("grey"))
).add_selection(states).properties(
    height=400,
    width=600
)


carrier_delay = alt.Chart(
    t[t.depdelay, t.arrdelay, t.carrier_name, t.origin_state],
    title="Carrier Departure Delay by Arrival Delay (Minutes)"
).transform_filter(
    states
).transform_aggregate(
    depdelay='mean(depdelay)',
    arrdelay='mean(arrdelay)',
    groupby=["carrier_name"]
).mark_point(filled=True, size=200).encode(
    x='depdelay',
    y='arrdelay',
    color=alt.condition(airlines, alt.ColorValue("steelblue"), alt.ColorValue("grey")),
    tooltip=['carrier_name', 'depdelay', 'arrdelay']
).add_selection(airlines).properties(
    height=400,
    width=600
)

# jupyterlab_omnisci.vega_ibis.FALLBACK = False

(flights_by_state | carrier_delay)

HConcatChart({
  hconcat: [Chart({
    data: Empty DataFrame
    Columns: [origin_state, carrier_name]
    Index: [],
    encoding: FacetedEncoding({
      color: ColorValue({
        condition: ColorValue({
          selection: 'selector005',
          value: 'steelblue'
        }),
        value: 'grey'
      }),
      x: X({
        aggregate: 'count',
        type: 'quantitative'
      }),
      y: Y({
        field: 'origin_state',
        sort: Sort({
          encoding: 'x',
          order: 'descending'
        }),
        type: 'nominal'
      })
    }),
    height: 400,
    mark: 'bar',
    selection: {'selector005': SelectionDef({
      fields: ['origin_state'],
      type: 'multi'
    })},
    title: 'Total Number of Flights by State',
    transform: [FilterTransform({
      filter: {'selection': 'selector006'}
    })],
    width: 600
  }), Chart({
    data: Empty DataFrame
    Columns: [depdelay, arrdelay, carrier_name, origin_state]
    Index: [],
    encoding: FacetedEnc

In [ ]:
jupyterlab_omnisci.vega_ibis.FALLBACK = True

# (flights_by_state | carrier_delay)